In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

cream_list = soup.find_all('div', class_='itemContainer')

url = []
for item in cream_list:
        for link in item.find_all('a', href=True): 
            url.append(link['href'])

In [4]:
creams_url = []
for i in range (1, 84):
    url = f'https://www.yesstyle.com/en/women-face-creams-moisturizers/list.html/bcc.14338_bpt.46#/bt=37&sb=136&bpt=46&pn={i}&s=10&bcc=14338&l=1' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    creams_list = soup.find_all('div', class_='itemContainer')
    for item in creams_list:
        for link in item.find_all('a', href=True): 
            creams_url.append(link['href'])


In [5]:
print(len(creams_url))

2988


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [51]:
"""
test_url = 'https://www.yesstyle.com/es/cosrx-gel-limpiador-con-ph-bajo-good-morning-150ml/info.html/pid.1052684732'
r = requests.get(test_url) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
name = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
for x in name:
    title = soup.find('h1').text
ingred = soup.find('div', class_="cropWrapper").text #ingredients
rating = soup.find('div', class_='satisfied').text
"""

In [6]:
creams_url[0]

'https://www.yesstyle.com/en/the-face-shop-real-nature-face-mask-1pc-20-types-20g-calendula/info.html/pid.1061020764'

In [6]:
creams = []

In [9]:
for link in creams_url[2269:2988]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name_box = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
        for x in name_box:
            name = soup.find('h1').text
        ingredients = soup.find('div', class_="cropWrapper").text.strip() #ingredients
        rating = soup.find('div', class_='satisfied').text.strip()
    except:
        name= 'no info'
        ingredients = 'no info'
        rating = 'no info'
        
    creams_info = {
    'name': name,
    'rating': rating,
    'ingredients': ingredients}
    
    creams.append(creams_info)

In [10]:
print(len(creams))

2988


In [12]:
creams[496]

IndexError: list index out of range

## 3. PASARLO A DATAFRAME

In [11]:
creams_df = pd.DataFrame(creams)
creams_df

,name,rating,ingredients
0,\niUNIK - Centella Calming Gel Cream 60ml\n,"96.7% customers satisfied|3,291 reviews","Centella Asiatica Leaf Water, Melaleuca Altern..."
1,"\nSOME BY MI - AHA, BHA, PHA 30 Days Miracle C...","95.4% customers satisfied|1,490 reviews",Centella Asiatica (Gotu Kola) Extract (70.78%)...
2,\nMIZON - Snail Repair Eye Cream 25ml\n,"95.9% customers satisfied|3,109 reviews","Snail Secretion Filtrate, Glycerin, Butylene G..."
3,\nRohto Mentholatum - Melano CC Vitamin C Esse...,"96.1% customers satisfied|1,842 reviews",Active ingredient : Ascorbic acid (Active vita...
4,\nMILKYDRESS - Wrinkle & Whitening Eye Cream 1...,"87.8% customers satisfied|2,737 reviews","Water, Butylene Glycol, Glycereth-26, Glycerin..."
...,...,...,...
2983,\nBenton - Aloe Propolis Soothing Gel 100ml\n,"96.6% customers satisfied|1,293 reviews","Aloe Barbadensis Leaf Water, Butylene Glycol, ..."
2984,\nNATURE REPUBLIC - Soothing & Moisture Aloe V...,"97.5% customers satisfied|3,557 reviews","Aloe Barbadensis Leaf Extract (92%), Alcohol, ..."
2985,\nROVECTIN - Skin Essentials Activating Treatm...,97% customers satisfied|278 reviews,"Water, Glycerin, Methyl Gluceth-20, Pentylene ..."
2986,\nCOSRX - Moisture Power Enriched Cream\n,97.7% customers satisfied|93 reviews,"Water, Glycerin, Ethylhexyl Stearate, Vinyl Di..."


In [12]:
creams = creams_df.to_csv('yesstyle_data/creams.csv')